In [1]:
import os
import sys

import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# dataset
src = o3d.io.read_point_cloud("src.ply")
dst = o3d.io.read_point_cloud("dst.ply")

Xall = np.asarray(src.points)
Yall = np.asarray(dst.points)

const_buf = np.ones((Xall.shape[0], 1))
Xall = np.concatenate((const_buf, Xall), axis=1) # 1+ local position
Yalls = np.hsplit(Yall, Yall.shape[1]) # independent global coordinate 

In [3]:
# recursive least squares

k = 3 # initial

X = Xall[:k]
# print(X)
# print(X.shape) # (k, 1+|features|)

dim_target = 0 # x, y, z
Y = Yalls[dim_target][:k] # TODO: do each RLS for each dimension!
# print(Y)
# print(Y.shape) # (k, 1)

P = np.linalg.inv(X.T @ X) # NOTE: inverse covariance matrix
# print(P)
# print(P.shape) # (1+|features|, 1+|features|)

beta = P @ X.T @ Y
# print(beta)
# print(beta.shape) # (1+|features|, 1), first dimension is intercept


for k in range(4, Xall.shape[0]):

    # print(k)

    z = Xall[k].reshape((Xall[k].shape[0], 1))
    y = Yalls[dim_target][k]
    

    G = (P @ z) / (1 + z.T @ P @ z) # weight
    # print(G)
    # print(G.shape) # (1, 1+|features|)
    
    P = P - ((G @ z.T) * P)
    # print(P)
    # print(P.shape) # (1+|features|, 1+|features|)

    beta = beta + G * (y - beta.T @ z)
    # print(beta)
    # print(beta.shape) # (1+|features|, 1)

    # break

print(beta)
idx_eval = 101
x_eval = Xall[idx_eval]
# print(x_eval)
# print(np.concatenate([x_eval[1:], x_eval[0]]))
# print(x_eval[1:])
# print(x_eval[0])
# print(np.array(x_eval[0]).reshape(1))
# print(np.concatenate([x_eval[1:], np.array(x_eval[0]).reshape(1)], axis=0))
x_eval = np.concatenate([x_eval[1:], np.array(x_eval[0]).reshape(1)], axis=0)

y_eval = Yalls[dim_target][idx_eval]
print(y_eval)
print(x_eval.dot(beta))


[[-29.96628819]
 [-37.67119207]
 [ 99.36610004]
 [-80.39318209]]
[-0.01247181]
[100.70198346]


In [4]:
# modulization check

from RLS_module import RLS

solver = RLS(X, Y)
# print(solver.beta)

for k in range(4, Xall.shape[0]):
    x = Xall[k]
    y = Yalls[dim_target][k]
    solver.add_new(x, y)

print(solver.beta)

idx_eval = 101
x_eval = Xall[idx_eval]
y_eval = Yalls[dim_target][idx_eval]
print(y_eval)
print(solver.evaluate(x_eval))

[[-29.96628819]
 [-37.67119207]
 [ 99.36610004]
 [-80.39318209]]
[-0.01247181]
[100.70198346]


In [5]:
# test with public data, 
# from https://github.com/adamDhalla/recursive-least-squares

# initialization
A = np.array([[1, 0], [1, 1], [1, 2]])
b = np.array([[3], [4], [7]])
solver = RLS(A, b)
print(f"x0={solver.beta}")

# update
newA = np.array([1, 3])
newb = np.array([11])
solver.add_new(newA, newb)
print(f"x1={solver.beta}")

x0=[[2.66666667]
 [2.        ]]
x1=[[2.2]
 [2.7]]


In [16]:
# centerize
Xm = np.mean(np.asarray(src.points), axis=0)
# print(Xm)
Xall_c = np.asarray(src.points)-np.tile(Xm, (Xall.shape[0], 1))
Xall_c = np.concatenate((const_buf, Xall_c), axis=1) # 1+ local position
# print(Xall)
# print(Xall_c) # NOTE: consider scaling as well


k_init=200 # increasing initial value somehow drops error, but is not wanted...
X_c = Xall_c[:k_init]
Y = Yalls[dim_target][:k_init] # TODO: do each RLS for each dimension!

from RLS_module import RLS

solver = RLS(X_c, Y)
# print(solver.beta)

for k in range(k_init, 500):
    x = Xall_c[k]
    y = Yalls[dim_target][k]
    try:
        solver.add_new(x, y)
    except FloatingPointError as e:
        print(f"{k=} {x=} {y=}")
        break

print(solver.beta)

idx_eval = 0
x_eval = Xall_c[idx_eval]
y_eval = Yalls[dim_target][idx_eval]
print(y_eval)
print(solver.evaluate(x_eval))

[[ 1.59159175]
 [ 0.84023324]
 [ 0.00618369]
 [-0.54244126]]
[0.27405298]
[-3.16006679]


In [43]:
# test learning multidimension


k_init=100
Xm = np.mean(np.asarray(src.points), axis=0)
Xall_c = np.asarray(src.points)-np.tile(Xm, (Xall.shape[0], 1))
Xall_c = np.concatenate((const_buf, Xall_c), axis=1)
X_c = Xall_c[:k_init]
XYZ = Yall[:k_init]


from RLS_module import RLS

solver = RLS(X_c, XYZ)

idx_eval = 0
x_eval = Xall_c[idx_eval]
y_eval = Yall[idx_eval]
print(y_eval)
print(solver.evaluate(x_eval))

# visualize
dst_RLS = o3d.geometry.PointCloud()
print(Xall_c.shape)
print(solver.beta.shape)

# np.concatenate([Xall_c[:, 1:], np.array(Xall_c[:, 0]).reshape(1)], axis=0)
print(Xall_c[:, 1:])
print(Xall_c[:, 1:].shape)
print(Xall_c[:, 0])
print(Xall_c[:, 0].shape)
# print(np.array(Xall_c[:, 0]).reshape((Xall_c.shape[0], 1)).shape)
# np.vstack((Xall_c[:, 1:], Xall_c[:, 0].reshape((Xall_c.shape[0], 1))))
x_eval_all = np.concatenate((np.asarray(src.points)-np.tile(Xm, (Xall.shape[0], 1)), const_buf), axis=1) # 1+ local position
print(x_eval_all)

dst_RLS.points = o3d.utility.Vector3dVector(x_eval_all.dot(solver.beta) + np.tile(Xm, (Xall.shape[0], 1)))
dst_RLS.colors = dst.colors
o3d.io.write_point_cloud("dst_RLS.ply", dst_RLS)

[0.27405298 1.02095513 1.12868408]
[-3.16006679 -1.60380728 -1.09813541]
(198835, 4)
(4, 3)
[[-1.11000265 -1.01789621  0.69792497]
 [-1.08656515 -1.01789621  0.6976558 ]
 [-1.07094015 -1.01789621  0.69787991]
 ...
 [-0.86344315  0.47429129  0.6823765 ]
 [-0.86781515  0.47553655  0.6823765 ]
 [-0.86781515  0.47429129  0.68432987]]
(198835, 3)
[1. 1. 1. ... 1. 1. 1.]
(198835,)
[[-1.11000265 -1.01789621  0.69792497  1.        ]
 [-1.08656515 -1.01789621  0.6976558   1.        ]
 [-1.07094015 -1.01789621  0.69787991  1.        ]
 ...
 [-0.86344315  0.47429129  0.6823765   1.        ]
 [-0.86781515  0.47553655  0.6823765   1.        ]
 [-0.86781515  0.47429129  0.68432987  1.        ]]


True